In [77]:
from PIL import Image
import requests
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch

In [78]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "mps"

processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
caption_model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b", torch_dtype=torch.float32
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [79]:
url = "https://lumiere-a.akamaihd.net/v1/images/h_101dalmatians_mobile_19752_992e2a21.jpeg?region=0%2C0%2C640%2C480"
image = Image.open(requests.get(url, stream=True).raw)
print(type(image))

inputs = processor(images=image, return_tensors="pt").to(torch.float32)

<class 'PIL.JpegImagePlugin.JpegImageFile'>


In [83]:
generated_ids = caption_model.generate(**inputs)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
print(generated_text)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[    2,   627,   385,   337,  9244,  2071,    32,  2828,   561,    11,
            10, 15800, 50118]])
the dalmatians are sitting together in a cartoon


torch.Size([1, 13])


In [81]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

In [82]:
# Sentences we want to encode. Example:
sentence = [generated_text, generated_text, generated_text]
passage_embedding = model.encode(['dalmatians',
                                  'cat'])

# Sentences are encoded by calling model.encode()
embedding = model.encode(sentence)
embedding = embedding.squeeze()
print(len(embedding))
print(len(passage_embedding[0]))

print("Similarity:", util.dot_score(embedding, passage_embedding))

3
384
Similarity: tensor([[0.7250, 0.2467],
        [0.7250, 0.2467],
        [0.7250, 0.2467]])
